**Import Library**

In [ ]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, asin, ceil
import time
import geocoder
from geopy.geocoders import Nominatim
import folium

**Pre-Processing**

In [ ]:
file_path = 'hotel-jatim.csv'
# Membaca dataset
df = pd.read_csv(file_path)

In [ ]:
df

,Unnamed: 0,Hotel Image,Hotel Name,City,Address,Hotel Rating,Max,Min,Facilities,Lattitute,Longitude
0,0,https://s-light.tiket.photos/t/01E25EBZS3W0FY9...,MaxoneHotels at Dharmahusada Surabaya,"Gubeng, Surabaya","Jl. Dharmahusada No.189, Mojo, Gubeng, Kota SB...",4.2,358500,1175880,"['WiFi', 'Parkir', 'Restoran', 'Resepsionis 24...",-7.267435,112.771124
1,1,https://s-light.tiket.photos/t/01E25EBZS3W0FY9...,D Carol Hotel Surabaya,"Bubutan, Surabaya","Jl. Gundih No. 8, Gundih, Kec. Bubutan, Kota S...",4.2,197000,586075,"['InDOnesia CARE', 'WiFi', 'Parkir', 'Restoran...",-7.247228,112.729460
2,2,https://s-light.tiket.photos/t/01E25EBZS3W0FY9...,Choice City Hotel,"Bubutan, Surabaya","Bubutan, Surabaya City, East Java 60174, Indon...",4.4,270000,375000,"['InDOnesia CARE', 'tiket CLEAN', 'WiFi', 'Par...",-7.255990,112.733686
3,3,https://s-light.tiket.photos/t/01E25EBZS3W0FY9...,G Suites Hotel By AMITHYA,"Gubeng, Surabaya","Jl. Raya Gubeng No.43, Gubeng, Kec. Gubeng, Ko...",3.9,287000,1886000,"['InDOnesia CARE', 'tiket CLEAN', 'Kolam Renan...",-7.273623,112.747475
4,4,https://s-light.tiket.photos/t/01E25EBZS3W0FY9...,Royal Regantris Cendana formerly Royal Singosa...,"Tegalsari, Surabaya","Jl. Kombes Pol. Moh. Duryat No.6, Tegalsari, K...",4.2,390000,1200000,"['InDOnesia CARE', 'tiket CLEAN', 'WiFi', 'Par...",-7.268379,112.739621
...,...,...,...,...,...,...,...,...,...,...,...
100,100,https://s-light.tiket.photos/t/01E25EBZS3W0FY9...,RedDoorz Syariah @ Griya Soeratin Pacitan,"Pacitan, Pacitan","Jl. KA. Posong No.1, Gemulung, Tanjungsari, Ke...",5.0,248724,275373,"['tiket CLEAN', 'WiFi', 'Parkir', 'Resepsionis...",-8.194593,111.112619
101,101,https://s-light.tiket.photos/t/01E25EBZS3W0FY9...,OYO 1190 Griya Soeratin Residence Syariah,"Pacitan, Pacitan","Jl. KA. Posong No.1, Gemulung, Tanjungsari, Pa...",3.8,166208,184016,"['tiket CLEAN', 'WiFi', 'Parkir', 'Resepsionis...",-8.194578,111.112639
102,102,https://s-light.tiket.photos/t/01E25EBZS3W0FY9...,VH Surf Camp,"Pacitan, Pacitan","Jl. Pandan Arum No 11, Pancer, Kelurahan Ploso,",Belum ada rating,100000,150000,"['Parkir', 'Pet-friendly', 'Parkir', 'Pet-frie...",-8.223100,111.101303
103,103,https://s-light.tiket.photos/t/01E25EBZS3W0FY9...,OYO 2643 Hotel Graha Prima,"Pacitan, Pacitan","Jl. Raya Tompak Rinjing No.10, Balong, Sidohar...",3.4,79572,161112,"['tiket CLEAN', 'WiFi', 'Parkir', 'Restoran', ...",-8.217934,111.074011


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    105 non-null    int64  
 1   Hotel Image   105 non-null    object 
 2   Hotel Name    105 non-null    object 
 3   City          105 non-null    object 
 4   Address       105 non-null    object 
 5   Hotel Rating  105 non-null    object 
 6   Max           105 non-null    int64  
 7   Min           105 non-null    int64  
 8   Facilities    105 non-null    object 
 9   Lattitute     105 non-null    float64
 10  Longitude     105 non-null    float64
dtypes: float64(2), int64(3), object(6)
memory usage: 9.1+ KB


In [ ]:
# format coordinates
def format_coordinate(value):
    try:
        value = float(value)
        if -90 <= value <= 90 or -180 <= value <= 180:
            return round(value, 6)
        else:
            return np.nan
    except ValueError:
        return np.nan
df['Lattitute_cleaned'] = df['Lattitute'].apply(format_coordinate)
df['Longitude_cleaned'] = df['Longitude'].apply(format_coordinate)

cleaned_data = df[['Hotel Name', 'Lattitute_cleaned', 'Longitude_cleaned']]
cleaned_data.to_csv('cleaned_hotel_data.csv', index=False)

print("Completed")

Completed


In [ ]:
cleaned_data

,Hotel Name,Lattitute_cleaned,Longitude_cleaned
0,MaxoneHotels at Dharmahusada Surabaya,-7.267435,112.771124
1,D Carol Hotel Surabaya,-7.247228,112.729460
2,Choice City Hotel,-7.255990,112.733686
3,G Suites Hotel By AMITHYA,-7.273623,112.747475
4,Royal Regantris Cendana formerly Royal Singosa...,-7.268379,112.739621
...,...,...,...
100,RedDoorz Syariah @ Griya Soeratin Pacitan,-8.194593,111.112619
101,OYO 1190 Griya Soeratin Residence Syariah,-8.194578,111.112639
102,VH Surf Camp,-8.223100,111.101303
103,OYO 2643 Hotel Graha Prima,-8.217934,111.074011


**Fungsi Haversine**

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = [radians(titik) for titik in [lon1, lat1, lon2, lat2]]
    selisih_lon = lon2 - lon1
    selisih_lat = lat2 - lat1
    a = sin(selisih_lat / 2)**2 + cos(lat1) * cos(lat2) * sin(selisih_lon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius Bumi dalam kilometer
    return c * r

**Latitude dan Longitude User**

In [ ]:
print('Latitude dan longitude berdasarkan :\n1. Alamat IP anda saat ini\n2. Nama tempat/alamat yang anda inginkan\n')

while True :
    input_lokasi = input('Anda pilih 1 atau 2? ')

    if input_lokasi == '1' :
        g = geocoder.ip('me')
        lat1 = g.latlng[0]
        lon1 = g.latlng[1]
        print(f"\nAlamat IP anda saat ini :\nLatitude: {lat1}, Longitude: {lon1}")

        break


    elif input_lokasi == '2' :

        while True :
            lokasi_user = input("\nMasukkan nama tempat atau alamat: ")

            geolocator = Nominatim(user_agent="maps_project_unesa")
            lokasi = geolocator.geocode(lokasi_user)

            if lokasi:
                lat1, lon1 = lokasi.latitude, lokasi.longitude
                print(f"Koordinat lokasinya: {lat1}, {lon1}")

                break

            else:
                print("Lokasi tidak ditemukan, silakan coba lagi.")
                continue

        break

    else :
        print('Harap mengetik angka 1 atau 2\n')

        continue

# map_ = folium.Map(location=[lat1, lon1], zoom_start=15)

# # Menambahkan marker lokasi
# folium.Marker(location=[lat1, lon1], popup="Lokasi Anda", tooltip="Klik untuk detail").add_to(map_)

# map_.save("lokasi_anda.html")
# print("\nPeta lokasi telah disimpan sebagai 'lokasi_anda.html'. Buka file ini di browser untuk melihat lokasi Anda.")

# map_

Latitude dan longitude berdasarkan :
1. Alamat IP anda saat ini
2. Nama tempat/alamat yang anda inginkan

Anda pilih 1 atau 2? 2

Masukkan nama tempat atau alamat: unesa ketintang
Koordinat lokasinya: -7.31475245, 112.72594360167578


**Baca Dataset**

In [ ]:
baca = pd.read_csv('cleaned_hotel_data.csv')

**Brute Force**

In [ ]:
jarak_ke_setiap_hotel = []
hotel_terdekat = []
durasi_brute_force = []

A = 0
while A < 5 :

    start_time = time.perf_counter()

    for i in range(len(baca)):
        lat2 = baca.iloc[i, 1]
        lon2 = baca.iloc[i, 2]

        a = haversine(lon1, lat1, lon2, lat2)
        jarak_ke_setiap_hotel.append((i,a))

    while True:
        terdekat = jarak_ke_setiap_hotel[0]

        for j in jarak_ke_setiap_hotel:
            if j[1] < terdekat[1]:
                terdekat = j

        hotel_terdekat.append(terdekat)
        jarak_ke_setiap_hotel.remove(terdekat)

        if jarak_ke_setiap_hotel == []:
            break

    end_time = time.perf_counter()

    durasi_brute_force.append(end_time - start_time)

    A += 1

print("Hotel-hotel terdekat:")
for k in range(min(5, len(hotel_terdekat))):
    hotel_index = hotel_terdekat[k][0]
    jarak = hotel_terdekat[k][1]
    print(f"{k+1}. {baca.iloc[hotel_index, 0]} - Jarak: {jarak:.2f} km")

total_durasi_brute_force = sum(durasi_brute_force)
ratarata_durasi_brute_force = sum(durasi_brute_force)/len(durasi_brute_force)

print(f'\nTotal durasi brute force 5 kali = {total_durasi_brute_force:.6f} detik')
print(f'Rata-rata durasi brute force 5 kali = {ratarata_durasi_brute_force:.6f} detik')

Hotel-hotel terdekat:
1. Hersya Front One Inn Surabaya - Jarak: 0.55 km
2. Hotel Oval Surabaya - Jarak: 2.63 km
3. ASTON Inn Jemursari - Jarak: 2.65 km
4. BATIQA Hotel Darmo Surabaya - Jarak: 2.79 km
5. Arimbi Stay - Jarak: 3.02 km

Total durasi brute force 5 kali = 0.033699 detik
Rata-rata durasi brute force 5 kali = 0.006740 detik


**Divide and Conquer (Merge Sort)**

In [ ]:
durasi_merge_sort = []

B = 0
while B < 5 :

    start_time = time.perf_counter()

    def merge_sort(arr):
        if len(arr) <= 1:
            return arr

        mid = len(arr) // 2
        left = arr[:mid]
        right = arr[mid:]

        left = merge_sort(left)
        right = merge_sort(right)

        return merge(left, right)

    def merge(left, right):
        result = []
        i = j = 0

        while i < len(left) and j < len(right):
            if left[i][1] < right[j][1]:
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1

        while i < len(left):
            result.append(left[i])
            i += 1
        while j < len(right):
            result.append(right[j])
            j += 1

        return result

    end_time = time.perf_counter()

    durasi_merge_sort.append(end_time - start_time)

    B += 1

jarak_ke_setiap_hotel = []

for i in range(len(baca)):
    lat2 = baca.iloc[i, 1]
    lon2 = baca.iloc[i, 2]

    b = haversine(lon1, lat1, lon2, lat2)
    jarak_ke_setiap_hotel.append((i, b))

sorted_hotels = merge_sort(jarak_ke_setiap_hotel)

print("Hotel-hotel terdekat:")
for j in range(min(5, len(sorted_hotels))):
    hotel_index = sorted_hotels[j][0]
    jarak = sorted_hotels[j][1]
    print(f"{j+1}. {baca.iloc[hotel_index, 0]} - Jarak: {jarak:.2f} km")

total_durasi_merge_sort = sum(durasi_merge_sort)
ratarata_durasi_merge_sort = sum(durasi_merge_sort)/len(durasi_merge_sort)

print(f'\nTotal durasi merge sort 5 kali = {total_durasi_merge_sort:.6f} detik')
print(f'Rata-rata durasi merge sort 5 kali = {ratarata_durasi_merge_sort:.6f} detik')

Hotel-hotel terdekat:
1. Hersya Front One Inn Surabaya - Jarak: 0.55 km
2. Hotel Oval Surabaya - Jarak: 2.63 km
3. ASTON Inn Jemursari - Jarak: 2.65 km
4. BATIQA Hotel Darmo Surabaya - Jarak: 2.79 km
5. Arimbi Stay - Jarak: 3.02 km

Total durasi merge sort 5 kali = 0.000007 detik
Rata-rata durasi merge sort 5 kali = 0.000001 detik


**Decrease and Conquer (Insertion Sort)**

In [ ]:
durasi_insertion_sort = []

C = 0
while C < 5 :

    start_time = time.perf_counter()

    def insertion_sort(arr):
            for i in range(1, len(arr)):
                key = arr[i]
                j = i - 1

                while j >= 0 and arr[j][1] > key[1]:
                    arr[j + 1] = arr[j]
                    j -= 1

                arr[j + 1] = key

            return arr

    end_time = time.perf_counter()

    durasi_insertion_sort.append(end_time - start_time)

    C += 1

jarak_ke_setiap_hotel = []

for i in range(len(baca)):
    lat2 = baca.iloc[i, 1]
    lon2 = baca.iloc[i, 2]

    c = haversine(lon1, lat1, lon2, lat2)
    jarak_ke_setiap_hotel.append((i, c))

hotel_terurut = insertion_sort(jarak_ke_setiap_hotel)

print("Hotel-hotel terdekat:")
for j in range(min(5, len(hotel_terurut))):
    hotel_index = hotel_terurut[j][0]
    jarak = hotel_terurut[j][1]
    print(f"{j+1}. {baca.iloc[hotel_index, 0]} - Jarak: {jarak:.2f} km")

total_durasi_insertion_sort = sum(durasi_insertion_sort)
ratarata_durasi_insertion_sort = sum(durasi_insertion_sort)/len(durasi_insertion_sort)

print(f'\nTotal durasi insertion sort 5 kali = {total_durasi_insertion_sort:.6f} detik')
print(f'Rata-rata durasi insertion sort 5 kali = {ratarata_durasi_insertion_sort:.6f} detik')


Hotel-hotel terdekat:
1. Hersya Front One Inn Surabaya - Jarak: 0.55 km
2. Hotel Oval Surabaya - Jarak: 2.63 km
3. ASTON Inn Jemursari - Jarak: 2.65 km
4. BATIQA Hotel Darmo Surabaya - Jarak: 2.79 km
5. Arimbi Stay - Jarak: 3.02 km

Total durasi insertion sort 5 kali = 0.000003 detik
Rata-rata durasi insertion sort 5 kali = 0.000001 detik


**Perbandingan Ketiga Algoritma**

In [ ]:
data = {
    'algoritma': ['brute force', 'merge sort', 'insertion sort'],
    'durasi rata-rata': [ratarata_durasi_brute_force, ratarata_durasi_merge_sort, ratarata_durasi_insertion_sort]
}

df = pd.DataFrame(data)
df_sorted = df.sort_values(by='durasi rata-rata')

tercepat_pertama = df_sorted.iloc[0,1]
tercepat_kedua = df_sorted.iloc[1,1]
tercepat_ketiga = df_sorted.iloc[2,1]

persentase1 = (tercepat_kedua-tercepat_pertama)/tercepat_kedua*100
persentase2 = (tercepat_ketiga-tercepat_pertama)/tercepat_ketiga*100

print(f'{df_sorted.iloc[0,0]} menjadi algoritma tercepat, yakni {persentase1:.2f}% lebih\ncepat dari {df_sorted.iloc[1,0]} dan {persentase2:.2f}% lebih cepat dari {df_sorted.iloc[2,0]}')

df_sorted

insertion sort menjadi algoritma tercepat, yakni 48.50% lebih
cepat dari merge sort dan 99.99% lebih cepat dari brute force


,algoritma,durasi rata-rata
2,insertion sort,6.700000e-07
1,merge sort,1.301000e-06
0,brute force,6.739709e-03


**Peta Visualisasi**

In [ ]:
peta = folium.Map(location=[lat1, lon1], zoom_start=10)

folium.Marker(
    location=[lat1, lon1],
    popup="Lokasi Anda",
    icon=folium.Icon(color="blue", icon="user"),).add_to(peta)

warna_garis = ["red", "green", "blue", "purple", "orange"]
ikon_hotel = ["home", "home", "home", "home", "home"]

for idx, (hotel_index, jarak) in enumerate(sorted_hotels[:5]):
    hotel_data = baca.iloc[hotel_index]
    hotel_lat = hotel_data['Lattitute_cleaned']
    hotel_lon = hotel_data['Longitude_cleaned']
    hotel_name = hotel_data['Hotel Name']

    folium.Marker(
        location=[hotel_lat, hotel_lon],
        popup=f"Hotel: {hotel_name}\nJarak: {jarak:.2f} km",
        icon=folium.Icon(color=warna_garis[idx % len(warna_garis)], icon=ikon_hotel[idx % len(ikon_hotel)]),).add_to(peta)

    folium.PolyLine(
        locations=[[lat1, lon1], [hotel_lat, hotel_lon]],
        color=warna_garis[idx % len(warna_garis)],
        weight=2.5,
        opacity=0.8,).add_to(peta)

    folium.Circle(
        location=[hotel_lat, hotel_lon],
        radius=500,  # Radius dalam meter
        color=warna_garis[idx % len(warna_garis)],
        fill=True,
        fill_opacity=0.1,).add_to(peta)

print("Visualisasi Jarak Lokasi dengan Hotel-hotel Terdekat")
peta.save("peta_hotel_terdekat.html")
peta

Visualisasi Jarak Lokasi dengan Hotel-hotel Terdekat
